In [1]:
from gensim.summarization import summarize
from newspaper import Article
from model import Model
from utils import build_dict, build_dataset, batch_iter
from rouge import Rouge
import pickle
import pandas as pd
import re
import tensorflow as tf
import random

import nltk 
from nltk.tag import pos_tag
from nltk import FreqDist
from nltk.tokenize import word_tokenize

import sym_vocab

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package words to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# 지문 생성 및 문제 생성을 위한 텍스트 파일 생성 
def crawl_news(url, raw_txt_name, num_words):
    # URL 기반 크롤링, 언어는 영어로 설정
    news = Article(url, language = 'en') # URL 가져오기
    news.download() # 해당 url의 html을 가져오기
    # HTML 파싱 적용
    news.parse() 
    # 지정된 단어 갯수를 기준으로 textrank를 적용하여 파싱된 텍스트를 요약 (Rule base)
    text = summarize(news.text, word_count = num_words) # 600자 기준으로 하기
    title = news.title # 파싱한 html에서 title에 해당하는 부분 추출하기
    title = title.lower() # title에 대해서 전부 소문자화 
    
    title = re.sub('\([^)]*\)', '', title) # 괄호를 포함한 문자열 제거 
    
    # 빈칸으로 만들 지문을 textrank 기준으로 선별
    blank_sentence = summarize(news.text, word_count = 30)
    
    # 크롤링한 raw data를 별도로 저장
    with open(raw_txt_name, 'w', encoding = 'utf-8') as f: # 크롤링한 raw data를 저장하기
        f.write(text)

    return text, title, blank_sentence

In [3]:
# 빈칸 지문 생성을 위한 함수를 별도로 정의
def bf_create_blank(raw_txt_name, prepro_txt_name):
    # 문장 번호: 문장 형식의 저장을 위해 변수 생성
    test_dict = {}
    
    # 저장했었던 크롤링 본문 데이터 불러오기
    with open(raw_txt_name, 'r', encoding = 'utf-8') as f:
    # raw data 상태에서 전처리를 통해 모델에 들어갈 수 있는 데이터셋으로 변형
        for idx, text in enumerate(f.readlines()):
            # print(idx, text)
            text = text.lower() # 소문자로 통일
            text = re.sub('[.]', '', text) # 일단 온점 모두 제거
            text = re.sub('[,]', ' ,', text) # 반점 하나의 단어로 인식
            text = re.sub("\’s",' \'s', text) # 소유격도 하나의 단어로 인식
            text = re.sub("[0-9]", '#', text) # 숫자를 #로 치환
            #text = re.sub("'.+'", '', text) # 작은 따옴표 안 고유명사 제거 
            text = re.sub('[“”]', '', text) # 큰 따옴표 제거 
            text = re.sub('\n', '', text) # 줄 바꿈표 제거 
            print(idx, text)
        
            # 사전에 저장하기
            test_dict[idx] = text ## {1: '텍스트', 2: '텍스트'}
            
    # 전처리된 데이터를 별도의 파일로 저장
    with open(prepro_txt_name, 'w', encoding = 'utf-8') as f: # 전처리한 데이터를 저장하기 
        # 줄 바꿈을 기준으로 문장을 구분하여 저장
        for i in range(len(test_dict)):
            test_dict[i] = test_dict[i] + " " + "." + '\n'
            f.write(test_dict[i])
    
    return test_dict 

In [4]:
# 텍스트 요약 학습 데이터 불러오기
def shake_bot_blank(filename, vaild_article_path):
    # 설정된 세팅 값 불러오기
    with open("args.pickle", "rb") as f:
        args = pickle.load(f)
        
    # 실행시 최초의 상태로 초기화하여 모델 재사용이 가능하도록 함
    tf.reset_default_graph()
    # 학습된 단어 사전과 본문 최대 길이와 요약 최대 길이 설정 값을 불러옴
    print("셰봇이 단어 사전 불러오는 중...")
    word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", args.toy)
    # 불러온 설정 값에 따라서 검증 데이터셋을 생성
    print("셰봇이 기본 설정 값 초기화 중...")
    valid_x = build_dataset("valid", vaild_article_path, word_dict, article_max_len, summary_max_len, args.toy)
    valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]

    with tf.Session() as sess:
        # 학습된 체크포인트와 모델을 불러오기 
        print("Loading saved model...")
        model = Model(reversed_dict, article_max_len, summary_max_len, args, forward_only=True) # 검증 단계
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state("./saved_model/")
        saver.restore(sess, ckpt.model_checkpoint_path)
        
        # 검증을 진행할 배치 사이즈 설정
        batches = batch_iter(valid_x, [0] * len(valid_x), args.batch_size, 1)
        
        # 검증 단계인 만큼 인코더 인풋 부분만 고려하여 학습 진행
        print("셰봇이 수능 빈칸 문제 생성 준비 중.. {}...".format(filename))
        for batch_x, _ in batches:
            # 문장 -> 문장 예측이 가능하도록 데이터 분할 
            batch_x_len = [len([y for y in x if y != 0]) for x in batch_x]
            # 검증을 위한 배치에 따라 모델 및 데이터 셋 설정 준비 
            valid_feed_dict = {
                model.batch_size: len(batch_x),
                model.X: batch_x,
                model.X_len: batch_x_len,
            }
            # Prediction을 목적으로 학습된 모델을 검증을 위한 설정 값을 적용하여 예측 결과 생성
            prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
            # 인덱스 -> 단어 사전을 통해 학습된 결과를 단어로 표현.
            prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]
            
            # 생성된 예측 결과물을 문장 단위로 저장 
            with open(filename, "w") as f:
                for line in prediction_output:
                    summary = list()
                    for word in line:
                        if word == "</s>":
                            break
                        if word not in summary:
                            summary.append(word)
                    print(" ".join(summary), file=f)
                    
            #with open('./rouge_test/sys_dir/{}.txt'.format(filename), "w") as f:
                #for line in prediction_output:
                    #summary = list()
                    #for word in line:
                        #if word == "</s>":
                            #break
                        #if word not in summary:
                            #summary.append(word)
                    #print(" ".join(summary), file=f)

        print('셰봇이가 5지 선다로 쓸만한 친구들을 추려냈어요.. {}...'.format(filename))

In [5]:
## textrank 없이 rouge score로 학습 완성도가 높은 문장을 선별 
def for_rouge_test(summy, label):
    
    rouge_dict = {}
    rouge_list = []
    rouge = Rouge()
    
    # 학습 결과로 예측한 결과물을 불러오기 
    with open(summy, 'r', encoding = 'utf-8') as f:
        
        for idx, line in enumerate(f.readlines()):
            
            # 실제 문장 - 예측 결과에서 학습의 정도를 Rouge-r과 Rouge-p로 검증
            scores = rouge.get_scores(line, label[idx])
            
            p_score = scores[0]["rouge-1"]["p"]
            r_score = scores[0]["rouge-1"]["r"]
            
            r_idx_score = (idx, r_score)
            p_idx_score = (idx, p_score)
            
            # Rouge-r 사용
            rouge_list.append(r_idx_score)
            #rouge_list.append(p_idx_score)
            
            rouge_dict[idx] = line 
        
        # 내림차순으로 가장 점수가 높은 문장을 순서로 리스트 배열
        sort_rouge_list = sorted(rouge_list, key = lambda rouge_list: rouge_list[-1], reverse=True)
        # 상위 5개 문장 추출하기 
        sort_rouge_list = sort_rouge_list[:5]
        
        question_list = []
    
        
        for ix in sort_rouge_list:
            
            question_list.append(rouge_dict[ix[0]])
        
    # 상위 5개 문장에 대해서 선지로 활용할 수 있도록 리턴  
    return question_list, sort_rouge_list

In [6]:
## textrank로 중요도가 높은 문장을 사전 추출하고 제작
def for_rouge_test2(summy, label, blank_sentence):
    rouge_dict = {}
    rouge_list = []
    rouge = Rouge()
    
    # 학습 결과로 예측한 결과물을 불러오기 
    with open(summy, 'r', encoding = 'utf-8') as f:
            
        for idx, line in enumerate(f.readlines()):
            
            # 실제 문장 - 빈칸 지문에서 학습의 정도를 Rouge-r과 Rouge-p로 검증
            scores = rouge.get_scores(label[idx], blank_sentence)
            
            print(blank_sentence)
            
            p_score = scores[0]["rouge-1"]["p"] 
            r_score = scores[0]["rouge-1"]["r"] 
            
            r_idx_score = (idx, r_score)
            p_idx_score = (idx, p_score)
            
            # Rouge-r score 사용
            rouge_list.append(r_idx_score)
            #rouge_list.append(p_idx_score)
            #print(rouge_list)
            
            rouge_dict[idx] = line
        
        # 내림차순으로 가장 점수가 높은 문장을 순서로 리스트 배열
        sort_rouge_list = sorted(rouge_list, key = lambda rouge_list: rouge_list[-1], reverse=True)
        # 상위 5개 문장 추출하기
        sort_rouge_list = sort_rouge_list[:5]
        
        
        question_list = []
    
        
        for ix in sort_rouge_list:
            
            question_list.append(rouge_dict[ix[0]])
        
    # 상위 5개 문장에 대해서 선지로 활용할 수 있도록 리턴
    return question_list, sort_rouge_list
            
            

In [7]:
# 학습 결과물 중 <unk> 부분 처리
def delete_unk(content, unk_sentence):
    
    container = []
    
    text = open(content, 'r', encoding = 'utf-8').read()
    
    # 각 단어에 대해서 품사 태깅
    tagged_list = pos_tag(word_tokenize(text))
    
    # 명사 부분만 활용
    nnp_list = [t[0] for t in tagged_list if (t[1] == "NNP") or (t[1] == "NN")]
    
    fd_names = FreqDist(nnp_list)
    
    # 빈도수 기반으로 Unk 처리하기
    freq = fd_names.most_common(2)
    
    #for tagged in tagged_list:
        
        #if tagged[1] == 'NN' or 'NNP':
            
            #container.append(tagged[0]) # 단어만 추출
    
    
    # unk 부분을 처리하는 방법 중 하나로 최고 빈도수를 기반으로 하는 방식
    ## 현재 여기서는 정확도를 해치는 것으로 판단 
    #replace_sentence = re.sub('< unk >', freq[0][0], unk_sentence)
    
    # 다중 숫자에 대한 표현은 Several로 대체 
    replace_sentence = re.sub('# nd', 'Several', unk_sentence)
    
    #replace_sentence = re.sub('# nd', 'Several', replace_sentence)
    
    # 불필요한 줄 바꿈표 제거 
    replace_sentence = re.sub('\n', '', replace_sentence)
    
    return replace_sentence
     


In [8]:
# 본문에 빈칸 만들기
def make_blank(idx, text): # 1개만 넣어야 함.
    # 줄바꿈 단위로 문장을 나누기 
    text_split = text.split('\n')
    print(text_split)
    # 선정한 문장 <-> 빈칸 교체
    text_split[idx] = '_______________________________.'
    text = '\n'.join(text_split)
    
    return text

In [9]:
# 본문에 동사 이후 빈칸 만들기 (S + V + O 라면, O 이후)
def make_blank_verb(idx, text, blank_sent): # 1개만 넣어야 함.
    # 줄바꿈 단위로 문장을 나누기 
    text_split = text.split('\n')
    print(text_split)
    text_split[idx] = blank_sent + ' _______________________________.'
    text = '\n'.join(text_split)
    
    return text

In [10]:
# 본문에 주어에 해당하는 명사 이후 빈칸 만들기 
def make_blank_noun(idx, text, blank_sent): # 1개만 넣어야 함.
    # 줄바꿈 단위로 문장을 나누기 
    text_split = text.split('\n')
    print(text_split)
    text_split[idx] = blank_sent + ' _______________________________.'
    text = '\n'.join(text_split)
    
    return text

In [11]:
# 문장 빈칸 문제 생성
url = 'https://www.nytimes.com/2019/11/14/world/australia/police-shooting-murder-Indigenous.html'
raw_txt_name = 'CSAT_31.txt'
prepro_txt_name = 'prepro_31.txt'
learned_txt_name = 'blank_31.txt'
num_words = 400

def blank_31(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, textrank = False):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기 
    text, title, blank_sentence = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    
    print(blank_sentence)
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_blank(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_blank(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    
    # Textrank를 적용한 Rouge Score에 따라 선지 생성 
    if textrank == True:
    
        question_list, sort_rouge_list = for_rouge_test2(learned_txt_name, test_dict, blank_sentence)
    
        print(question_list)
    
    # Textrank를 적용하지 않은 Rouge Score에 따라 선지 생성
    if textrank == False:
        
        question_list, sort_rouge_list = for_rouge_test(learned_txt_name, test_dict)
    
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        if idx == 0: # 첫 번째는 항상 빈칸 지문임. 
            
            replace_sentence = replace_sentence.capitalize() + "."   
    
        if idx > 0: # 나머지는 다른 지엽정 정보를 요약한 형태 
            
            token = word_tokenize(replace_sentence)
            
            tag = pos_tag(token)
            
            # 동사, 명사, 형용사에서 유사어로 치환
            v_list = [t[0] for t in tag if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ") or
                     (t[1] == "NN") or (t[1] == "NNS") or (t[1] == "NNP") or (t[1] == "NNPS") or (t[1] == "JJ") or (t[1] == "JJR") or (t[1] == "JJS") or (t[1] == "RB")]
            
            # 유사어 치환
            for v in v_list:
                
                paraphrasing_dict = sym_vocab.crawling_dict(v)
                
                paraphrasing_list = paraphrasing_dict[v]
                
                # 유사어가 존재하는 경우 랜덤하게 1가지 추출
                if paraphrasing_list == list:
                
                    random.shuffle(paraphrasing_list)
             
                    token[token.index(v)]= paraphrasing_list[0]
                
                # 유사어가 존재하지 않는 경우 최초 입력값 그대로 다시 반환
                elif paraphrasing_list != list:
                    
                    token[token.index(v)] = paraphrasing_list
                    
            replace_sentence = ' '.join(token)
            
            #replace_sentence = re.sub(" \’s",'\'s', replace_sentence)
                
            replace_sentence = replace_sentence.capitalize() + "."
            
            print(replace_sentence)
        
        # 빈칸 선지 1개, 지엽적 선지 4개 저장
        question_list[idx] = replace_sentence
       
    # 빈칸으로 설정한 인덱스 추적 -> 해당 문장 번호를 빈칸으로 대체 
    text = make_blank(sort_rouge_list[0][0], text)
        
    text = re.sub('\n', ' ', text) # raw data에서 불필요한 줄바꿈표 제거 (상대적일듯)
    
    # 정답 빈칸이 무조건 1번이 였으므로, 랜덤하게 섞어주기
    random.shuffle(question_list)
    
    number_list = ['①', '②', '③', '④', '⑤']
    
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        # 랜덤하게 섞은 선지에 넘버링 
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('31. 다음 빈칸에 들어갈 말로 가장 적절한 것을 고르시오.')
    print('-----------------------------------')
    print(text)
    print('\n')
    for question in question_list:
        print(question)
    
    
    
    
    
    
    

if __name__ == "__main__":
    
    blank_31(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, textrank = True)
    

Mr. Walker did not receive medical care after he was shot on Saturday, because staff had evacuated the clinic in Yuendumu earlier in the day over safety concerns.
0 sydney , australia — thousands of australians have taken to the streets this week to protest the police killing of an aboriginal teenager in a remote community , a case that has added fuel to long-simmering anger over the government 's behavior toward the country 's indigenous people
1 on wednesday , the police officer who fatally shot the ##-year-old man , kumanjayi walker , in the central australian town of yuendumu was charged with murder
2 but questions have continued to swirl about what happened when officers were alone with mr walker , as well as how he and his family were treated in the hours that followed
3 mr walker did not receive medical care after he was shot on saturday , because staff had evacuated the clinic in yuendumu earlier in the day over safety concerns
4 that left any medical emergencies to be handled 

In [12]:
# 문장 빈칸 문제 생성
url = 'https://www.nytimes.com/2019/11/14/technology/whistleblower-name-facebook-youtube.html?action=click&module=Top%20Stories&pgtype=Homepage'
raw_txt_name = 'CSAT_32.txt'
prepro_txt_name = 'prepro_32.txt'
learned_txt_name = 'blank_32.txt'
num_words = 300

def blank_32(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, textrank = False):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기 
    text, title, blank_sentence = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    
    print(blank_sentence)
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_blank(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_blank(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    
    # Textrank를 적용한 Rouge Score에 따라 선지 생성 
    if textrank == True:
    
        question_list, sort_rouge_list = for_rouge_test2(learned_txt_name, test_dict, blank_sentence)
    
        print(question_list)
    
    # Textrank를 적용하지 않은 Rouge Score에 따라 선지 생성
    if textrank == False:
        
        question_list, sort_rouge_list = for_rouge_test(learned_txt_name, test_dict)
    
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        if idx == 0: # 첫 번째는 항상 빈칸 지문임. 
            
            replace_sentence = replace_sentence.capitalize() + "."   
    
        if idx > 0: # 나머지는 다른 지엽정 정보를 요약한 형태 
            
            token = word_tokenize(replace_sentence)
            
            tag = pos_tag(token)
            
            # 동사, 명사, 형용사에서 유사어로 치환
            v_list = [t[0] for t in tag if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ") or
                     (t[1] == "NN") or (t[1] == "NNS") or (t[1] == "NNP") or (t[1] == "NNPS") or (t[1] == "JJ") or (t[1] == "JJR") or (t[1] == "JJS") or (t[1] == "RB")]
            
            # 유사어 치환
            for v in v_list:
                
                paraphrasing_dict = sym_vocab.crawling_dict(v)
                
                paraphrasing_list = paraphrasing_dict[v]
                
                # 유사어가 존재하는 경우 랜덤하게 1가지 추출
                if paraphrasing_list == list:
                
                    random.shuffle(paraphrasing_list)
             
                    token[token.index(v)]= paraphrasing_list[0]
                
                # 유사어가 존재하지 않는 경우 최초 입력값 그대로 다시 반환
                elif paraphrasing_list != list:
                    
                    token[token.index(v)] = paraphrasing_list
                    
            replace_sentence = ' '.join(token)
            
            #replace_sentence = re.sub(" \’s",'\'s', replace_sentence)
                
            replace_sentence = replace_sentence.capitalize() + "."
            
            print(replace_sentence)
        
        # 빈칸 선지 1개, 지엽적 선지 4개 저장
        question_list[idx] = replace_sentence
       
    # 빈칸으로 설정한 인덱스 추적 -> 해당 문장 번호를 빈칸으로 대체 
    text = make_blank(sort_rouge_list[0][0], text)
        
    text = re.sub('\n', ' ', text) # raw data에서 불필요한 줄바꿈표 제거 (상대적일듯)
    
    # 정답 빈칸이 무조건 1번이 였으므로, 랜덤하게 섞어주기
    random.shuffle(question_list)
    
    number_list = ['①', '②', '③', '④', '⑤']
    
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        # 랜덤하게 섞은 선지에 넘버링 
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('32. 다음 빈칸에 들어갈 말로 가장 적절한 것을 고르시오.')
    print('-----------------------------------')
    print(text)
    print('\n')
    for question in question_list:
        print(question)
    
    
    
    
    
    
    

if __name__ == "__main__":
    
    blank_32(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, textrank = False)

On Instagram, photographs purporting to show the whistle-blower with a number of public figures from the Democratic Party were also widely shared.
0 youtube had taken additional steps to make it difficult to search for the name , removing an autocomplete feature that filled in the name once a person began typing it
1 on tuesday morning , the top three videos claiming to share the whistle-blower 's name each had over ### ,### views
2 instead , they shared it in the comments and discussions below the video , where people also linked to blogs discussing the identity
3 many videos with the name were taken down after the new york times asked youtube about them
4 facebook was unwilling to comment on whether it was seeing a coordinated effort to spread the whistle-blower 's name , but said it would continue to take down content that violated its policies
5 youtube did not respond to a request for comment on the videos being shared on its site
6 on instagram , photographs purporting to show th

In [14]:
# 동사 이후 빈칸 만들기 
url = 'https://www.nytimes.com/2019/11/01/style/self-care/fish-oil-benefits.html'
raw_txt_name = 'CSAT_33.txt'
prepro_txt_name = 'prepro_33.txt'
learned_txt_name = 'blank_33.txt'
num_words = 500

def blank_33(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, textrank = False):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인 추출하기
    text, title, blank_sentence = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_blank(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_blank(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    
    # Textrank를 적용한 Rouge Score에 따라 선지 생성 
    if textrank == True:
        
        question_list, sort_rouge_list = for_rouge_test2(learned_txt_name, test_dict, blank_sentence)
    
    # Textrank를 적용하지 않은 Rouge Score에 따라 선지 생성
    if textrank == False:
        
        question_list, sort_rouge_list = for_rouge_test(learned_txt_name, test_dict)
    
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
            
            # ans_sent = replace_sentence # 동사 빈칸 문제 생성을 위해 정답 선지 사전 대입 
        
        # 정답 선지를 제외한 선지에 대해 전처리 적용 (이 부분도 재검토)
        if idx > 0:
            
            token = word_tokenize(replace_sentence)
            
            tag = pos_tag(token)
            
            # 동사, 명사, 형용사에서 유사어로 치환
            v_list = [t[0] for t in tag if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ") or
                     (t[1] == "NN") or (t[1] == "NNS") or (t[1] == "NNP") or (t[1] == "NNPS") or (t[1] == "JJ") or (t[1] == "JJR") or (t[1] == "JJS") or (t[1] == "RB")]
            
            # 유사어 치환
            for v in v_list:
                
                paraphrasing_dict = sym_vocab.crawling_dict(v)
                
                paraphrasing_list = paraphrasing_dict[v]
                
                # 유사어가 존재하는 경우 랜덤하게 1가지 추출
                if paraphrasing_list == list:
                
                    random.shuffle(paraphrasing_list)
             
                    token[token.index(v)]= paraphrasing_list[0]
                
                # 유사어가 존재하지 않는 경우 최초 입력값 그대로 다시 반환
                elif paraphrasing_list != list:
                    
                    token[token.index(v)] = paraphrasing_list
                    
            replace_sentence = ' '.join(token)
            
        # 빈칸 선지 1개, 지엽적 선지 4개 저장
        question_list[idx] = replace_sentence
    
    
    for i, q in enumerate(question_list): # 선지 뽑아내기 
    
        token_q = word_tokenize(q)
    
        tag_q = pos_tag(token_q)
        
        
        for ix, t in enumerate(tag_q):
            # 동사 유무 판단, 동사 위치를 인덱스 값으로 저장
            if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ"):
                
                idx_num = ix
                
                break
            
        # 해당 문장에 동사가 이상 없이 존재하여, 문장에서 최초의 동사를 찾는다면
        if ix == len(tag_q) - 1:
            # 선지 리스트에 넣기
            question_list[i] = q 
            
            break
                
        # 첫번째 원소의 경우 빈칸 정답으로 활용. 
        elif i == 0: 
            
            save_sent = token_q[:idx_num + 1] # 동사 이전 문장까지 살리기
            
            save_sent = ' '.join(save_sent)
            
            save_sent = save_sent.capitalize()
            
            abandon_sent = token_q[idx_num + 1:] # 동사 이후 문장 선지화 
            
            abandon_sent = ' '.join(abandon_sent) 
            
            question_list[i] = abandon_sent # 동사 이후 문장은 선지에 넣기 
            
            
        # 나머지 4개 원소의 경우 선지로 활용
        else:
            
            abandon_sent = token_q[idx_num + 1:] # 동사 이후 문장 선지화 
            
            abandon_sent = ' '.join(abandon_sent) 
        
            question_list[i] = abandon_sent # 동사 이후 문장은 선지에 넣기 
            
    # 빈칸으로 설정한 인덱스 추적 -> 해당 문장 번호를 빈칸으로 대체 
    text = make_blank_verb(sort_rouge_list[0][0], text, blank_sent = save_sent)
        
    text = re.sub('\n', ' ', text) # raw data에서 불필요한 줄바꿈표 제거 (상대적일듯)

        
        
    # 정답 빈칸이 현재 무조건 1번이므로, 랜덤하게 섞어주기 
    random.shuffle(question_list)
    
    number_list = ['①', '②', '③', '④', '⑤']
    
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('33. 다음 빈칸에 들어갈 말로 가장 적절한 것을 고르시오.')
    print('-----------------------------------')
    print(text)
    print('\n')
    for question in question_list:
        print(question)
    
    
    
    
    
    
    

if __name__ == "__main__":
    
    blank_33(url, raw_txt_name, prepro_txt_name, learned_txt_name,num_words, textrank = False)

0 african-americans benefited regardless of fish intake , showing a ## percent lower risk of heart attack
1 this could be a chance finding , said dr joann manson , a director of the study and the chief of the division of preventive medicine at brigham and women 's hospital
2 we do plan to pursue it in greater detail and try to replicate it in a separate trial because if this can be reproduced , that would be a very dramatic benefit to african-americans
3 because there is still more research to be done , experts don’t necessarily recommended that african-americans take omega-#
4 if you have some history of heart disease or high triglycerides ( an estimated ## percent of adults in the united states do , according to data from the national health and nutrition examination survey in #### ) , it may be a good idea to take omega-#
5 the potential downside , because supplements are not regulated , is that production isn’t standardized so we don’t know what 's in them , according to dr pieter 

In [15]:
# 주어 이후 서술형 빈칸 문제 생성
url = 'https://www.nytimes.com/2019/11/14/us/politics/trump-tax-returns-supreme-court.html'
raw_txt_name = 'CSAT_34.txt'
prepro_txt_name = 'prepro_34.txt'
learned_txt_name = 'blank_34.txt'
num_words = 500

def blank_34(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, textrank = False):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기 
    text, title, blank_sentence = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_blank(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_blank(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    
    # Textrank를 적용한 Rouge Score에 따라 선지 생성 
    if textrank == True:
    
        question_list, sort_rouge_list = for_rouge_test2(learned_txt_name, test_dict, blank_sentence)
    
    # Textrank를 적용하지 않은 Rouge Score에 따라 선지 생성
    elif textrank == False:
        
        question_list, sort_rouge_list = for_rouge_test(learned_txt_name, test_dict)
    
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
            
            # ans_sent = replace_sentence # 동사 빈칸 문제 생성을 위해 정답 선지 사전 대입 
            
        # 정답 선지를 제외한 선지에 대해 전처리 적용 (이 부분도 재검토)
        if idx > 0:
            
            token = word_tokenize(replace_sentence)
            
            tag = pos_tag(token)
            
            # 동사, 명사, 형용사에서 유사어로 치환
            v_list = [t[0] for t in tag if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ") or
                     (t[1] == "NN") or (t[1] == "NNS") or (t[1] == "NNP") or (t[1] == "NNPS") or (t[1] == "JJ") or (t[1] == "JJR") or (t[1] == "JJS") or (t[1] == "RB")]
            
            # 유사어 치환
            for v in v_list:
                
                paraphrasing_dict = sym_vocab.crawling_dict(v)
                
                paraphrasing_list = paraphrasing_dict[v]
                
                # 유사어가 존재하는 경우 랜덤하게 1가지 추출
                if paraphrasing_list == list:
                
                    random.shuffle(paraphrasing_list)
             
                    token[token.index(v)]= paraphrasing_list[0]
                
                # 유사어가 존재하지 않는 경우 최초 입력값 그대로 다시 반환
                elif paraphrasing_list != list:
                    
                    token[token.index(v)] = paraphrasing_list
                    
            replace_sentence = ' '.join(token)
        
        # 빈칸 선지 1개, 지엽적 선지 4개 저장
        question_list[idx] = replace_sentence
    
    for i, q in enumerate(question_list): # 선지 뽑아내기 
        
        token_q = word_tokenize(q)
    
        tag_q = pos_tag(token_q)
        
        # print(tag_q)
    
        for ix, t in enumerate(tag_q):
            # 동사 유무 판단, 동사 위치를 인덱스 값으로 저장
            if (t[1] == "NN") or (t[1] == "NNS") or (t[1] == "NNP") or (t[1] == "NNPS"):
                
                idx_num = ix
                
                break
            
        # 해당 문장에 동사가 이상 없이 존재하여, 문장에서 최초의 동사를 찾는다면
        if ix == len(tag_q) - 1:
            # 선지 리스트에 넣기    
            question_list[i] = q 
            
            break
                
        # 첫번째 원소의 경우 빈칸 정답으로 활용. 
        elif i == 0:
            
            save_sent = token_q[:idx_num + 1] # 동사 이전 문장까지 살리기
            
            save_sent = ' '.join(save_sent)
            
            save_sent = save_sent.capitalize()
            
            abandon_sent = token_q[idx_num + 1:] # 동사 이후 문장 선지화 
            
            abandon_sent = ' '.join(abandon_sent) 
            
            question_list[i] = abandon_sent # 동사 이후 문장은 선지에 넣기 
            
        # 나머지 4개 원소의 경우 선지로 활용
        else:
            
            abandon_sent = token_q[idx_num + 1:] # 동사 이후 문장 선지화 
            
            abandon_sent = ' '.join(abandon_sent) 
        
            question_list[i] = abandon_sent # 동사 이후 문장은 선지에 넣기 
             
    # 빈칸으로 설정한 인덱스 추적 -> 해당 문장 번호를 빈칸으로 대체 
    text = make_blank_verb(sort_rouge_list[0][0], text, blank_sent = save_sent)
        
    text = re.sub('\n', ' ', text) # raw data에서 불필요한 줄바꿈표 제거 (상대적일듯)
        
    # 정답 빈칸이 현재 무조건 1번이므로, 랜덤하게 섞어주기 
    random.shuffle(question_list)
    
    number_list = ['①', '②', '③', '④', '⑤']
    
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('34. 다음 빈칸에 들어갈 말로 가장 적절한 것을 고르시오.')
    print('-----------------------------------')
    print(text)
    print('\n')
    for question in question_list:
        print(question)
    
    
    
    
    
    
    

if __name__ == "__main__":
    
    blank_34(url, raw_txt_name, prepro_txt_name, learned_txt_name,num_words, textrank = True)

0 last week  , a unanimous three-judge panel of a federal appeals court in manhattan ruled against mr trump
1 the court , in a focused ruling , said state prosecutors may require third parties to turn over a sitting president 's financial records for use in a grand jury investigation 
2 mr trump has fought vigorously to shield his financial records , and prosecutors in manhattan have agreed not to seek the tax returns until the case is resolved by the supreme court
3 in exchange , they insisted on a very quick briefing schedule , one that would allow the court to announce whether it would hear the case as soon as next month and to issue a decision by june , as the presidential election enters its final stages
4 other cases involving mr trump are also in the pipeline
5 they involve matters as diverse as demands from house democrats for tax and business records , a request for access to redacted portions of the report prepared by robert s
6 mueller iii , the special counsel , and challen